# Jiaming Lin

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## 1 Is COMPAS fair? (60pt)

#### 1. (2pt) Load the COMPAS data, and perform a basic sanity checks.

In [2]:
cs = pd.read_csv("compas-score-two-years-reduced-370.csv.bz2")
print(cs.shape)
print(cs.isna().sum())
cs.head(5)

(6172, 14)
Unnamed: 0                 0
age                        0
c_charge_degree            0
race                       0
age_cat                    0
score_text                 0
sex                        0
priors_count               0
days_b_screening_arrest    0
decile_score               0
is_recid                   0
two_year_recid             0
c_jail_in                  0
c_jail_out                 0
dtype: int64


,Unnamed: 0,age,c_charge_degree,race,age_cat,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,c_jail_in,c_jail_out
0,1,69,F,Other,Greater than 45,Low,Male,0,-1,1,0,0,2013-08-13 06:03:42,2013-08-14 05:41:20
1,2,34,F,African-American,25 - 45,Low,Male,0,-1,3,1,1,2013-01-26 03:45:27,2013-02-05 05:36:53
2,3,24,F,African-American,Less than 25,Low,Male,4,-1,4,1,1,2013-04-13 04:58:34,2013-04-14 07:02:04
3,4,44,M,Other,25 - 45,Low,Male,0,0,1,0,0,2013-11-30 04:50:18,2013-12-01 12:28:56
4,5,41,F,Caucasian,25 - 45,Medium,Male,14,-1,6,1,1,2014-02-18 05:08:24,2014-02-24 12:18:30


#### 2. (2pt) Filter the data to keep only only Caucasian and African-Americans.

In [3]:
cs = cs[(cs["race"] == "African-American") | (cs["race"] == "Caucasian")]

#### 3. (2pt) Create a new dummy variable based off of COMPAS' risk score (decile_score), which indicates if an individual was classified as low risk (score 1-4) or high risk (score 5-10).

In [4]:
cs["risk_level"] = cs.decile_score
cs["risk_level"] = pd.to_numeric(cs.risk_level)
cs["risk_level"].loc[(cs["risk_level"] <= 4)] = 0
cs["risk_level"].loc[(cs["risk_level"] >= 5)] = 1 

#### 4. (6pt) Now analyze the offenders across this new risk category:

#### (a) What is the recidivism rate for low-risk and high-risk individuals?

#### (b) What are the recidivism rates for African-Americans and Caucasians?

In [5]:
#(a)
low = cs[cs.risk_level == 0]
print(low.shape)

low_re = low[low.two_year_recid == 1]
print(low_re.shape)

print("The recidivism rate for low-risk is " + str(881/2753))

high = cs[cs.risk_level == 1]
print(high.shape)

high_re = high[high.two_year_recid == 1]
print(high_re.shape)

print("The recidivism rate for high-risk is " + str(1602/2525))

#b
cau = cs[cs.race == "Caucasian"]
print(cau.shape)

cau_re = cau[cau.two_year_recid == 1]
print(cau_re.shape)

print("The recidivism rate for Caucasian is " + str(822/2103))

aa = cs[cs.race == "African-American"]
print(aa.shape)

aa_re = aa[aa.two_year_recid == 1]
print(aa_re.shape)

print("The recidivism rate for Aferican-American is " + str(1661/3175))

# The recidivism rate for low-risk is 0.3200
# The recidivism rate for high-risk is 0.6345
# The recidivism rate for Caucasian is 0.3909
# The recidivism rate for Aferican-American is 0.5231

(2753, 15)
(881, 15)
The recidivism rate for low-risk is 0.3200145296040683
(2525, 15)
(1602, 15)
The recidivism rate for high-risk is 0.6344554455445545
(2103, 15)
(822, 15)
The recidivism rate for Caucasian is 0.3908701854493581
(3175, 15)
(1661, 15)
The recidivism rate for Aferican-American is 0.5231496062992126


#### 5. (10 pt) Now create a confusion matrix comparing COMPAS predictions for recidivism (low risk/high risk) and the actual two-year recidivism and interpret the results.

In [6]:
# Total: 5278

# low risk and no recidivism: 1872
print(2753-881)
# high risk and no recidivism: 923
print(2525-1602)

1872
923


Confusion Matrix for COMPAS: No for Low risk(no recidivists), Yes for High risk(recidivists)

|            | No(predict) | Yes(predict)               |
| :---        |    :----:   |          ---: |
| No(actual)        | 1872         | 923           |
| Yes(actual)        | 881      | 1602               |

The accuracy is 0.6582, which means that 65.82% of individuals. The precision is 0.6345, which means that the percentage of correctness is 63.45% when individuals are predicted recidivists. The recall is 0.6452, which means that the for all recidivists, it predicted correct 64.52% of recidivists. F1 score is 0.6398.
33.02% not recidivists were falsly classifed as high-risk, and 35.48% recidivists were falsly classifed as low-risk.

#### 6. (12pt) Note the accuracy of the COMPAS classification, and also how its errors were distributed. Would you feel comfortable having a judge to use COMPAS to inform sentencing guidelines? At what point would the error/misclassification risk be acceptable for you?


I don't feel comfortable because its accuracy, precision and recall are both not good. If COMPAS classification plays an important role in judges, I will accept to use it at least all those values are 75% correct.

#### 7. (14pt) Now, you repeat your confusion matrix calculation and analysis from 5. But this time do it separately for African-Americans and for Caucasians: (a) How accurate is the COMPAS classification for African-American individuals? For Caucasians? (b) What are the false positive rates FPR = FP/N = FP/(FP + TN)?(c) The false negative rates FNR = FN/P = FN/(FN + TP)?


In [7]:
cau = cs[cs.race == "Caucasian"]
print("Number of Caucasians")
print(cau.shape) 
cau_low = cau[cau.risk_level == 0]
cau_low_re = cau_low[cau_low.two_year_recid == 1]
print(cau_low_re.shape)
cau_low_nre = cau_low[cau_low.two_year_recid == 0]
print(cau_low_nre.shape)

cau_high = cau[cau.risk_level == 1]
cau_high_re = cau_high[cau_high.two_year_recid == 1]
print(cau_high_re.shape)
cau_high_nre = cau_high[cau_high.two_year_recid == 0]
print(cau_high_nre.shape)

Number of Caucasians
(2103, 15)
(408, 15)
(999, 15)
(414, 15)
(282, 15)


Confusion Matrix for Caucasians: No for not recidivists, Yes for recidivists

|            | No(predict) | Yes(predict)               |
| :---        |    :----:   |          ---: |
| No(actual)        | 999         | 282           |
| Yes(actual)        | 408      | 414               |

In [8]:
# Sorry, I just too lazy to assign new names
cau = cs[cs.race == "African-American"]
print("Number of Caucasians")
print(cau.shape) 
cau_low = cau[cau.risk_level == 0]
cau_low_re = cau_low[cau_low.two_year_recid == 1]
print(cau_low_re.shape)
cau_low_nre = cau_low[cau_low.two_year_recid == 0]
print(cau_low_nre.shape)

cau_high = cau[cau.risk_level == 1]
cau_high_re = cau_high[cau_high.two_year_recid == 1]
print(cau_high_re.shape)
cau_high_nre = cau_high[cau_high.two_year_recid == 0]
print(cau_high_nre.shape)

Number of Caucasians
(3175, 15)
(473, 15)
(873, 15)
(1188, 15)
(641, 15)


Confusion Matrix for African-American: No for not recidivists, Yes for recidivists

|            | No(predict) | Yes(predict)               |
| :---        |    :----:   |          ---: |
| No(actual)        | 873         | 641           |
| Yes(actual)        | 473      | 1188               |

The accuray for African-American is 0.6491, and for Caucasians is 0.6719. For African-American, the false positive rate is 0.42, and for Caucasians is 0.22. The false negative rate for African-American is 0.28, for Caucasians is 0.50

#### 8. (12pt) If you have done this correctly, you will find that COMPAS's true negative and true positive percentages are similar for African-American and Caucasian individuals, but that false positive rates and false negative rates are difierent. Look again at the overal recidivism rates in the dataset for Black and White individuals. In your opinion, is the COMPAS algorithm `fair'? Justify your answer.

I don't think it is fair because althoguh both accuracy do not have too much difference. The False positive rates and false negative rates have obvious difference. The false positive rate for African-American is much higher than Caucasians, which means that COMPAS is more likely to identify innocent African-Americans are crimes than innocent Caucasians. The false negative rate for African-American is much lower than Caucasians. which means that COMPAS is more likely to predicts low risk for Caucasians recidivists than African-American 
recidivists. If COMPAS is fair, false positive rates and false negative rates should has little difference between 2 races, but here it is clearly not, which means it is not algorithm fair.

## 2 Can you beat COMPAS? (40pt)

#### 1. (8pt) Before we start: what do you think, what is an appropriate model performance measure here? A, P, R, F or something else? Maybe you want to report multiple measures? Explain!

I want to use A and F. Accuracy is important since we will not use the model if the model cannot predict too much correct. F-score is also important here because it combines P and R, since for this situation we both want it can predict with high correct percentage when individuals are recidivists and innocent individuals.

#### 2. (6pt) Now it is time to do the modeling. Create a logistic regression model that contains all the (permissible) explanatory variables you have in data into the model. (Some of these you have to convert to dummies). Do not include the variables discussed above. We also do not include race and gender in this model to avoid explicit gender/racial bias. Use 10-fold CV to compute its relevant performance measure(s) you discussed above

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [10]:
X = cs[["c_charge_degree", "age_cat", "priors_count", "days_b_screening_arrest"]]
y = cs.two_year_recid

a = pd.get_dummies(X.c_charge_degree, drop_first=True)
b = pd.get_dummies(X.age_cat, drop_first=True)
x = pd.concat([a,b],axis =1)
X = X.drop(["c_charge_degree", "age_cat"], axis = 1)
X = pd.concat([x,X],axis =1)

m = LogisticRegression()
cv = cross_val_score(m, X, y, scoring="f1", cv=10)
print("F1: ",cv.mean())
# F1-score for this model is 0.6233

m = LogisticRegression()
cv = cross_val_score(m, X, y, scoring="accuracy", cv=10)
print("A: ",cv.mean())
# Accuracy for this model is 0.6631

F1:  0.6232789263104306
A:  0.6631253234431603


#### 3. (6pt) Experiment with difierent models to find the best model according to your preformance indicator. (Include/exclude difierent variables, you may also do feature engineering, and try other model categories). Still do not include race and gender. Report your best model's performance. Is it better or worse than for the COMPAS model?

In [11]:
from sklearn.tree import DecisionTreeClassifier

m = DecisionTreeClassifier()
cv = cross_val_score(m, X, y, scoring="f1", cv=10)
print("F1:", np.mean(cv)) 

m = DecisionTreeClassifier()
cv = cross_val_score(m, X, y, scoring="accuracy", cv=10)
print("A:", np.mean(cv)) 

F1: 0.5981089468281039
A: 0.6538478954631705


In [12]:
from sklearn.ensemble import RandomForestClassifier

m = RandomForestClassifier(n_estimators=100, n_jobs=6)
cv = cross_val_score(m, X, y, scoring="f1", cv=10)
print("F1:", np.mean(cv))

m = RandomForestClassifier(n_estimators=100, n_jobs=6)
cv = cross_val_score(m, X, y, scoring="accuracy", cv=10)
print("A:", np.mean(cv))

F1: 0.6071331876129936
A: 0.6527086567764936


My best model is logistic regression model with A = 0.6631 and f1 = 0.6233. I got a slightly higher accuracy and slightly lower f1-score, so it is kind of tied.

#### 4. (4pt) Now add sex to the model. Does it help to improve the performance?

In [13]:
sex = pd.get_dummies(cs.sex)

X = pd.concat([X, sex],axis =1)

m = LogisticRegression()
cv = cross_val_score(m, X, y, scoring="accuracy", cv=10)
print("A: ",cv.mean())

m = LogisticRegression()
cv = cross_val_score(m, X, y, scoring="f1", cv=10)
print("F1: ",cv.mean())

A:  0.6689940198953482
F1:  0.6243692225784991


Sex variable does not helped too much.

#### 5. (4pt) And finally add race. Does the model improve?

In [14]:
race = pd.get_dummies(cs.race)

X = pd.concat([X, race],axis =1)

m = LogisticRegression()
cv = cross_val_score(m, X, y, scoring="accuracy", cv=10)
print("A: ",cv.mean())

m = LogisticRegression()
cv = cross_val_score(m, X, y, scoring="f1", cv=10)
print("F1: ",cv.mean())

A:  0.6665318986832269
F1:  0.6237314405062644


Age variable does not help too much either.

#### 6. (12pt) Discuss the results. Did you manage to beat COMPAS? Do gender and race help to improve your predictions? What should judges do when having access to such models? Should they use such models?


I tied with COMPAS, but this is their failure to tie with me. Gender and race does not improve predictions. Judges should not rely on it too much since I don't think its accuracy is better than an impartial judge.